In [1]:
import sys
import os
import datetime as dt
import warnings
import numpy as np
import pandas as pd
warnings.filterwarnings("ignore")
project_root = os.path.abspath(os.path.join("..", "scraper"))
sys.path.insert(0, project_root)
from utils import database_connection_and_cursor

In [2]:
def database_table(table_name: str):
    connection, cursor = database_connection_and_cursor("CSGO")
    cursor.execute(f"SELECT * FROM {table_name}")
    data_fetched = cursor.fetchall()
    columns = [col_name[0] for col_name in cursor.description]
    data_df = pd.DataFrame(data_fetched, columns=columns).infer_objects()
    connection.close()
    return data_df

In [3]:
pp_projs = database_table("prizepicks_lines_proj")
pp_projs["game_date"] = pd.to_datetime(pp_projs["game_date"])

In [4]:
def results_by_days(data: pd.DataFrame, n_days: int = 1, specific_day: bool = True):
    # Define target date
    target_day = (dt.datetime.today() - dt.timedelta(days=n_days)).date()

    # Filter by specific day or range
    if (n_days == 1) | (specific_day):
        data = data[data["game_date"] == str(target_day)]
    else:
        data = data[data["game_date"] >= str(target_day)]

    # Define MAP level categorization based on 'stat_type' column
    def map_level(stat_type):
        if "MAPS 1-2" in stat_type:
            return "MAPS 1-2"
        elif "MAPS 1-3" in stat_type:
            return "MAPS 1-3"
        elif "MAPS 1" in stat_type:
            return "MAPS 1"
        elif "MAPS 3" in stat_type:
            return "MAPS 3"
        else:
            return "Unknown"

    # Apply MAP level categorization
    data["MAP Level"] = data["stat_type"].apply(map_level)

    # Initialize results container
    results_by_map = {}

    # Loop through each MAP level and calculate statistics
    for map_level in data["MAP Level"].unique():
        map_data = data[data["MAP Level"] == map_level]

        predicted_overs_and_unders = np.where(
            map_data["model_projection"] <= map_data["line_score"], "Under", "Over"
        )
        actual_overs_and_unders = np.where(
            map_data["actual_result"] <= map_data["line_score"], "Under", "Over"
        )

        # Calculate correct Over and Under predictions
        correct_overs = np.sum((predicted_overs_and_unders == "Over") & (actual_overs_and_unders == "Over"))
        correct_unders = np.sum((predicted_overs_and_unders == "Under") & (actual_overs_and_unders == "Under"))

        # Predicted counts
        predicted_overs = np.sum(predicted_overs_and_unders == "Over")
        predicted_unders = np.sum(predicted_overs_and_unders == "Under")

        # Percentages for Over and Under accuracy, relative to each prediction type
        correct_over_percentage_relative = (correct_overs / predicted_overs) * 100 if predicted_overs > 0 else 0
        correct_under_percentage_relative = (correct_unders / predicted_unders) * 100 if predicted_unders > 0 else 0

        # Overall accuracy for each MAP level
        map_results = map_data["hit"].value_counts() / len(map_data)
        overall_accuracy = round(map_results.get("Correct", 0) * 100, 2)

        # Store results for the current MAP level
        results_by_map[map_level] = {
            "Total Predictions": len(map_data),
            "Overall Accuracy (%)": overall_accuracy,
            "Over Hit Rate (%)": f"{round(correct_over_percentage_relative, 2)}% ({correct_overs}/{predicted_overs})",
            "Under Hit Rate (%)": f"{round(correct_under_percentage_relative, 2)}% ({correct_unders}/{predicted_unders})",
        }

    # Calculate overall metrics across all MAP levels
    predicted_overs_and_unders_all = np.where(
        data["model_projection"] <= data["line_score"], "Under", "Over"
    )
    actual_overs_and_unders_all = np.where(
        data["actual_result"] <= data["line_score"], "Under", "Over"
    )

    # Calculate correct Over and Under predictions for overall data
    correct_overs_all = np.sum((predicted_overs_and_unders_all == "Over") & (actual_overs_and_unders_all == "Over"))
    correct_unders_all = np.sum((predicted_overs_and_unders_all == "Under") & (actual_overs_and_unders_all == "Under"))

    # Predicted counts for overall data
    predicted_overs_all = np.sum(predicted_overs_and_unders_all == "Over")
    predicted_unders_all = np.sum(predicted_overs_and_unders_all == "Under")

    # Percentages for correct Over and Under predictions overall
    correct_over_percentage_relative_all = (correct_overs_all / predicted_overs_all) * 100 if predicted_overs_all > 0 else 0
    correct_under_percentage_relative_all = (correct_unders_all / predicted_unders_all) * 100 if predicted_unders_all > 0 else 0

    # Overall accuracy for the entire dataset
    overall_results = data["hit"].value_counts() / len(data)
    overall_accuracy_all = round(overall_results.get("Correct", 0) * 100, 2)

    # Store overall results
    results_by_map["Overall"] = {
        "Total Predictions": len(data),
        "Overall Accuracy (%)": overall_accuracy_all,
        "Over Hit Rate (%)": f"{round(correct_over_percentage_relative_all, 2)} ({correct_overs_all}/{predicted_overs_all})",
        "Under Hit Rate (%)": f"{round(correct_under_percentage_relative_all, 2)} ({correct_unders_all}/{predicted_unders_all})",
    }

    # Print the results
    for level, metrics in results_by_map.items():
        print(f"\nMAP Level: {level}")
        print(f"Total Predictions: {metrics['Total Predictions']}")
        print(f"Overall Accuracy: {metrics['Overall Accuracy (%)']}%")
        print(f"Over Hit Rate: {metrics['Over Hit Rate (%)']}")
        print(f"Under Hit Rate: {metrics['Under Hit Rate (%)']}")

    # Return data and results dictionary for further analysis if needed
    data = data.reset_index(drop=True)
    return data, results_by_map

In [5]:
print("Yesterday's Result")
data, res_map = results_by_days(data=pp_projs, n_days=1, specific_day=True)
data.to_csv("yesterdays_hit_rate.csv")

Yesterday's Result

MAP Level: MAPS 1-2
Total Predictions: 48
Overall Accuracy: 56.25%
Over Hit Rate: 61.9% (26/42)
Under Hit Rate: 16.67% (1/6)

MAP Level: MAPS 3
Total Predictions: 2
Overall Accuracy: 100.0%
Over Hit Rate: 100.0% (2/2)
Under Hit Rate: 0% (0/0)

MAP Level: Overall
Total Predictions: 50
Overall Accuracy: 58.0%
Over Hit Rate: 63.64 (28/44)
Under Hit Rate: 16.67 (1/6)


In [6]:
n_days = int(input("Enter the number of days you'd like to look back: "))
print(f"{n_days} Days Result")
data, res_map = results_by_days(data=pp_projs, n_days=n_days, specific_day=False)
data.to_csv(f"{n_days}day_hit_rate.csv")

7 Days Result

MAP Level: MAPS 1-2
Total Predictions: 158
Overall Accuracy: 60.76%
Over Hit Rate: 62.41% (83/133)
Under Hit Rate: 40.0% (10/25)

MAP Level: MAPS 3
Total Predictions: 28
Overall Accuracy: 64.29%
Over Hit Rate: 53.57% (15/28)
Under Hit Rate: 0% (0/0)

MAP Level: Overall
Total Predictions: 186
Overall Accuracy: 61.29%
Over Hit Rate: 60.87 (98/161)
Under Hit Rate: 40.0 (10/25)
